## 랭체인을 표현하는 언어, LCEL
LCEL(LangChanin Expression Language) : 쉬운 코드 작성과 효과적인 모듈 관리를 위한 언어 체계 / 여러 모듈을 엮어 하나의 파이프라인 즉, 체인(chain)을 간단하게 만들 수 있음
기본적인 체인 구성 : 프롬프트 템플릿 -> LLM -> 출력 파서

In [1]:
# LCEL로 기본 체인 구성 (프롬프트 템플릿 -> LLM -> 출력 파서)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

import os

api_key=os.environ["GEMINI_API_KEY"]

# 프롬프트 탬플릿 설정
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

# LLM 호출
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21", api_key=api_key)

# 출력 파서 설정
output_parser = StrOutputParser()

# LCEL로 프롬프트 템플릿 - LLM - 출력 파서 연결
chain = prompt | model | output_parser

# invoke 함수로 실행
chain.invoke({"topic" : "ice cream"})

'Why did the ice cream go to therapy?\n\nBecause it felt so broken!'

In [3]:
# stream() : 스트리밍 기능 추가

from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

import os 

api_key = os.environ["GEMINI_API_KEY"]

# chain 선언
model = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-thinking-exp-01-21", api_key=api_key)
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
chain = prompt | model 

# chain의 stream()함수를 통해 스트리밍 기능 추가 
for i in chain.stream({"topic" : "bears"}):
    print(i.content, end="", flush=True)

Why don't bears wear shoes?

Because they have bear feet!

In [10]:
# 여러 개 API 요청하고 받는 batch()
# 랭체인을 RAG외 다양한 서비스 접목하거나 자연어 처리를 위해 사용될 수 있다.
# 예를 들어 번역 작업을 할 때 여러 입력 값과 출력 값을 한 번에 얻고 싶다면 병렬 실행 기능이 필요한데, LCEL의 batch()를 활용한다. 
# 여러 개의 한글 문장을 불어 문장으로 번역하는 작업을 batch()로 한꺼번에 수행하는 코드 / 1개의 문장 번역 작업과 5개의 문장 번역 작업의 수행 시간 비교

# 5개의 문장 번역 batch()

import time 

start = time.time()
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21", api_key=api_key)
prompt = ChatPromptTemplate.from_template("다음 한글 문장을 프랑스어로 번역해줘{sentence}")
chain = prompt | model 

results = chain.batch([
    {"sentence" : "그녀는 매일 아침 책을 읽습니다."},
    {"sentence" : "오늘 날씨가 참 좋네요."},
    {"sentence" : "저녁에 친구들과 영화를 볼거에요."},
    {"sentence" : "그 학생은 매우 성실하게 공부합니다."},
    {"sentence" : "커피 한 잔이 지금 딱 필요해요."}
])
end = time.time()

for i, result in enumerate(results):
    print(f"번역 {i+1}: {result.content}")
    
duration = end - start
print(duration)

번역 1: "Elle lit des livres tous les matins."

**설명:**

* **그녀는 (geunyeoneun):**  Elle (엘) - 그녀, 그녀는
* **매일 아침 (maeil achim):** tous les matins (뚜 레 마탱) - 매일 아침
    * tous (뚜) - 모든, 매
    * les (레) - 정관사 복수
    * matins (마탱) - 아침들 (복수형으로 매일 아침을 나타냄)
* **책을 (chaegeul):** des livres (데 리브르) - 책들
    * des (데) - 부정관사 복수 (여기서는 '몇 권의' 보다는 일반적인 '책들'을 의미)
    * livres (리브르) - 책들 (복수)
* **읽습니다 (ikseumnida):** lit (리) - 읽는다 (lire 동사 3인칭 단수 현재형)

**직역:** 그녀는 매일 아침 책들을 읽는다.

**자연스러운 프랑스어 표현:** "Elle lit des livres tous les matins." (그녀는 매일 아침 책을 읽습니다.) 가 가장 자연스럽고 일반적인 표현입니다.
번역 2: "오늘 날씨가 참 좋네요."를 프랑스어로 번역하면 다음과 같습니다:

**Aujourd'hui, il fait très beau.**

**설명:**

* **Aujourd'hui:** 오늘 (Today)
* **il fait beau:** 날씨가 좋다 (The weather is nice/good).  프랑스어에서 날씨를 표현할 때 주로 "faire" 동사를 사용합니다. "Beau"는 "아름다운", "좋은"이라는 뜻으로 날씨를 묘사할 때 사용됩니다.
* **très:**  참, 매우 (Very).  "참" 또는 "정말"의 강조 의미를 나타냅니다.

**다른 표현 (뉘앙스 차이):**

* **Aujourd'hui, il fait beau.:**  오늘 날씨가 좋네요. (가장 기본적인 표현)
* **Aujourd'hui, il fait vraimen

In [8]:
# 1개의 문장 번역

import time 

start = time.time()
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-thinking-exp-01-21", api_key=api_key)
prompt = ChatPromptTemplate.from_template("다음 한글 문장을 프랑스어로 번역해줘{sentence}")
chain = prompt | model 

result = chain.invoke({"sentence" : "그녀는 매일 아침 책을 읽습니다."})
end = time.time()

duration = end - start
print(duration)
print(result)

7.74027156829834
content='"Elle lit un livre tous les matins."\n\n**설명:**\n\n* **Elle:**  "그녀는" (geunyeoneun) 에 해당하는 프랑스어 인칭 대명사 "she" 입니다.\n* **lit:** "읽습니다" (ilkseumnida) 에 해당하는 "lire" (읽다) 동사의 현재 시제 3인칭 단수 형태입니다.\n* **un livre:** "책을" (chaegeul) 에 해당하는 "a book" 입니다.  "livre" 는 "책" 이라는 명사이고, "un" 은 남성 단수 명사 앞에 붙는 부정관사 "a" 입니다.  여기서는 특정 책이 아닌 일반적인 책을 읽는다는 의미이므로 부정관사를 사용했습니다.\n* **tous les matins:** "매일 아침" (maeil achim) 에 해당하는 "every morning" 입니다. "tous les" 는 "every" 또는 "all" 이라는 의미이고, "matins" 는 "아침" 이라는 명사 "matin" 의 복수 형태입니다.  매일 아침이라는 반복적인 행위를 나타내기 위해 복수 형태를 사용했습니다.\n\n**다른 가능한 번역 (뉘앙스 차이):**\n\n* **Elle lit des livres tous les matins.** (책을 *여러 권* 읽는다는 뉘앙스가 강조될 수 있습니다. 복수 명사 "livres" 와 복수 부정관사 "des" 를 사용했습니다.)\n* **Elle lit le livre tous les matins.** (만약 특정 책을 매일 읽는다는 의미라면 "le livre" 라고 표현할 수 있습니다. 정관사 "le" 는 "the" 에 해당합니다.)\n* **Chaque matin, elle lit un livre.** (시간 표현 "Chaque matin" (매일 아침) 을 문장 맨 앞에 놓아 강조할 수 있습니다.)\n\n하지만 가장 일반적이고 자연스러운 번역은 **"Elle lit un livre tous les mati